# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

**04_imfdata.ipynb**

Using the IMF API, data on exports, prices, and exchange rates are collected for all available countries during 2008-2014.

METODO: Annotate and clean method2 section - can I use a loop?

#### Import packages

In [2]:
import requests                                             # For requesting json data from the url
import pandas as pd                                         # pandas dataframes used for convenience
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

#### settings for API request

In [3]:
webserv = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/' # the main URL for the JSON rest API
method = 'CompactData/'                                     # Data is stored here. Datastructure has info
series = 'IFS'                                              # International Financial Statistics series
freq = 'A'                                                  # Annual
inds = {'x': 'TXG_FOB_USD', 'q': 'EREER_IX', 'p': 'TMG_D_USD_CIF_IX'}  # Set of IFS indicators of interest 
date = '?startPeriod=2008&endPeriod=2014'                   # Date range of interest

Gather additional dictionaries with country and indicator codes. CLEAN UP and ANNOTATE

In [108]:
method2 = 'DataStructure/' # This method gives info on the country names, units, and indicator names
url2 = webserv+method2+series   # url to access IMF datastructure method API
data1 = requests.get(url2).json()['Structure']['CodeLists']['CodeList'][2]['Code'] # area names here
data2 = requests.get(url2).json()['Structure']['CodeLists']['CodeList'][3]['Code'] # indicator codes
data3 = requests.get(url2).json()['Structure']['CodeLists']['CodeList'][0]['Code'] # unit codes
df = pd.DataFrame(data1).set_index('@value')
area_names = {c : df['Description'].loc[c]['#text'] for c in df.index.values}
df = pd.DataFrame(data2).set_index('@value')
ifs_inds = {i : df['Description'].loc[i]['#text'] for i in df.index.values}
df = pd.DataFrame(data3).set_index('@value')
unit_codes = {m : df['Annotations'].loc[m]['Annotation'][2]['AnnotationText']['#text'] for m in df.index.values}

Print out the full indicator names

In [88]:
for k in inds.keys():                         # keys are the x, p, q values in inds
    print inds[k]+': '+ifs_inds[inds[k]]      # print the indicator id and name

EREER_IX: Real Effective Exchange Rate, based on Consumer Price Index, Index
TXG_FOB_USD: Goods, Value of Exports, Free on board (FOB), US Dollars
TMG_D_USD_CIF_IX: Goods, Deflator/Unit Value of Imports, US Dollars, Index


#### Loop with API request for each indicators

In [181]:
fd = {} # dictionary for saving each series from inds above
unit_mult = {} # dictionary for saving unit multipliers by country and indicator
for k, v in inds.iteritems(): # k is the key and v is the value
    url = webserv+method+series+'/'+freq+'..'+v+'.'+date # print url to see
    df = pd.DataFrame(requests.get(url).json()['CompactData']['DataSet']['Series']).set_index('@REF_AREA')
    df['@UNIT_MULT'] = df['@UNIT_MULT'].map(unit_codes) # match unit codes with unit multipliers
    df = df[df['Obs'].apply(lambda x: isinstance(x, list))] # drops empties
    d = {} # temporary dict to save country by country dataframes
    for c in df.index.values: # index values are countries (@REF_AREA) as set above
        d[c] = pd.DataFrame(df.loc[c]['Obs']).rename(columns={'@TIME_PERIOD':'date'})
        d[c]['date'] = pd.to_datetime(d[c]['date'].values) # set dates as datetime for index
        d[c]['@OBS_VALUE'] = pd.to_numeric(d[c]['@OBS_VALUE']) * int(df['@UNIT_MULT'][c]) # adjust units
    fd[k]= pd.concat(d, axis=0).reset_index().set_index(['level_0','date']).drop('level_1', 1)
merged = pd.concat(fd, axis=1).reset_index() # combine all series to one merged dataframe
merged['full_name'] = merged['level_0'].map(area_names)    # add column with full name of area
merged = merged.set_index(['level_0','full_name','date'])  # set index to country and date